In [1]:
import pandas as pd
import numpy as np

In [2]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import getpass
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from scipy.stats import chi2_contingency
import datetime
import warnings
from sklearn.decomposition import TruncatedSVD
from scipy.stats import chi2_contingency

from sklearn.svm import SVC
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import TomekLinks

In [3]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [4]:
from sklearn.ensemble import RandomForestClassifier

## The business question: 1. Are people going to donate or not?

Exploring the data

In [5]:
file = pd.read_csv('learningSet (1).csv')

/Users/genivabraaf/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
file.head()

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,,,3712,0,,...,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,BOA,1,CA,91326,,,5202,0,,...,0.0,0,L,2,G,X,X,X,1.0,A
2,9001,AMH,1,NC,27017,,,0,0,,...,0.0,1,L,4,E,X,X,X,60.0,C
3,8701,BRY,0,CA,95953,,,2801,0,,...,0.0,1,L,4,E,X,X,X,41.0,C
4,8601,,0,FL,33176,,,2001,0,X,...,0.0,1,L,2,F,X,X,X,26.0,A


In [7]:
file.shape

(95412, 481)

In [8]:
file.describe()

,ODATEDW,TCODE,DOB,AGE,NUMCHLD,INCOME,WEALTH1,HIT,MBCRAFT,MBGARDEN,...,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,TARGET_B,TARGET_D,HPHONE_D,RFA_2F,CLUSTER2
count,95412.000000,95412.000000,95412.000000,71747.000000,12386.000000,74126.000000,50680.000000,95412.000000,42558.000000,42558.000000,...,95412.000000,85439.000000,85439.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95280.000000
mean,9141.363256,54.223117,2723.602933,61.611649,1.527773,3.886248,5.345699,3.321438,0.152075,0.059166,...,9135.651648,9151.022917,8.093739,13.347786,95778.176959,0.050759,0.793073,0.500618,1.910053,31.533711
std,343.454752,953.844476,2132.241295,16.664199,0.806861,1.854960,2.742490,9.306899,0.470023,0.262078,...,320.394019,294.257260,8.213242,10.769997,55284.596094,0.219506,4.429725,0.500002,1.072749,18.764614
min,8306.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,7211.000000,0.000000,1.285714,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,8801.000000,0.000000,201.000000,48.000000,1.000000,2.000000,3.000000,0.000000,0.000000,0.000000,...,8810.000000,8903.000000,4.000000,8.384615,47910.750000,0.000000,0.000000,0.000000,1.000000,15.000000
50%,9201.000000,1.000000,2610.000000,62.000000,1.000000,4.000000,6.000000,0.000000,0.000000,0.000000,...,9201.000000,9204.000000,6.000000,11.636364,95681.500000,0.000000,0.000000,1.000000,2.000000,32.000000
75%,9501.000000,2.000000,4601.000000,75.000000,2.000000,5.000000,8.000000,3.000000,0.000000,0.000000,...,9409.000000,9409.000000,11.000000,15.477955,143643.500000,0.000000,0.000000,1.000000,3.000000,49.000000
max,9701.000000,72002.000000,9710.000000,98.000000,7.000000,7.000000,9.000000,241.000000,6.000000,4.000000,...,9603.000000,9702.000000,1088.000000,1000.000000,191779.000000,1.000000,200.000000,1.000000,4.000000,62.000000


In [9]:
file.isna().sum().sort_values(ascending = False).head(50)

RDATE_5     95403
RAMNT_5     95403
RDATE_3     95170
RAMNT_3     95170
RDATE_4     95131
RAMNT_4     95131
RAMNT_6     94636
RDATE_6     94636
RAMNT_15    88150
RDATE_15    88150
RDATE_23    87553
RAMNT_23    87553
RDATE_20    87524
RAMNT_20    87524
RAMNT_7     86517
RDATE_7     86517
RAMNT_17    86011
RDATE_17    86011
RDATE_21    85899
RAMNT_21    85899
RAMNT_10    84951
RDATE_10    84951
RDATE_13    83162
RAMNT_13    83162
NUMCHLD     83026
RDATE_11    80672
RAMNT_11    80672
RDATE_19    79535
RAMNT_19    79535
RAMNT_9     78678
RDATE_9     78678
RDATE_24    77674
RAMNT_24    77674
RAMNT_18    75634
RDATE_18    75634
RAMNT_22    74539
RDATE_22    74539
RAMNT_8     73940
RDATE_8     73940
RDATE_14    72095
RAMNT_14    72095
RAMNT_12    69712
RDATE_12    69712
RAMNT_16    68418
RDATE_16    68418
ADATE_15    65477
ADATE_23    56270
MBCOLECT    52914
MAGFAML     52854
MBCRAFT     52854
dtype: int64

In [10]:
df_numerical = file.select_dtypes(include='int64')

In [11]:
df_cat = file.select_dtypes(include='object')

In [12]:
df_numerical 

,ODATEDW,TCODE,DOB,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,...,NGIFTALL,CARDGIFT,MINRDATE,MAXRDATE,LASTDATE,FISTDATE,CONTROLN,TARGET_B,HPHONE_D,RFA_2F
0,8901,0,3712,0,0,39,34,18,10,2,...,31,14,9208,9402,9512,8911,95515,0,0,4
1,9401,1,5202,16,0,15,55,11,6,2,...,3,1,9310,9512,9512,9310,148535,0,0,2
2,9001,1,0,2,0,20,29,33,6,8,...,27,14,9111,9207,9512,9001,15078,0,1,4
3,8701,0,2801,2,0,23,14,31,3,0,...,16,7,8711,9411,9512,8702,172556,0,1,4
4,8601,0,2001,60,1,28,9,53,26,3,...,37,8,9310,9601,9601,7903,7112,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,9601,1,0,0,14,36,47,11,7,8,...,1,0,9602,9602,9602,9602,184568,0,0,1
95408,9601,1,5001,1,0,31,43,19,4,1,...,1,0,9603,9603,9603,9603,122706,0,1,1
95409,9501,1,3801,0,0,18,46,20,7,23,...,7,4,9603,9501,9610,9410,189641,0,1,3
95410,8601,0,4005,0,0,28,35,20,9,1,...,41,18,9011,9608,9701,8612,4693,1,1,4


In [13]:
df_cat

,OSOURCE,STATE,ZIP,MAILCODE,PVASTATE,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,...,RFA_21,RFA_22,RFA_23,RFA_24,RFA_2R,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,GRI,IL,61081,,,0,,,,,...,S4E,S4E,S4E,S4E,L,E,X,X,X,C
1,BOA,CA,91326,,,0,,,,,...,N1E,N1E,,F1E,L,G,X,X,X,A
2,AMH,NC,27017,,,0,,,,,...,,S4D,S4D,S3D,L,E,X,X,X,C
3,BRY,CA,95953,,,0,,,,,...,A1D,A1D,,,L,E,X,X,X,C
4,,FL,33176,,,0,X,X,,,...,A3D,I4E,A3D,A3D,L,F,X,X,X,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,ASE,AK,99504,,,0,,,,,...,,,,,L,G,X,X,X,C
95408,DCD,TX,77379,,,0,,,,,...,,,,,L,F,X,X,X,A
95409,MBC,MI,48910,,,0,,X,,,...,P1D,P1D,,,L,E,X,X,X,B
95410,PRV,CA,91320,,,0,X,,,,...,A1F,A1F,S2F,S3F,L,F,X,X,X,A


In [14]:
file_corr = file.corr()['TARGET_B'].sort_values(ascending=False)
file_corr

TARGET_B    1.000000
TARGET_D    0.774232
RFA_2F      0.072311
CARDGIFT    0.054027
NGIFTALL    0.050896
              ...   
RAMNT_3    -0.095351
RDATE_3    -0.126060
RAMNT_5    -0.380296
ADATE_5          NaN
ADATE_15         NaN
Name: TARGET_B, Length: 407, dtype: float64

In [15]:
nulls = pd.DataFrame(file.isna().sum()*100/len(file), columns=['percentage'])
nulls.sort_values('percentage', ascending = False).head(80)

,percentage
RDATE_5,99.990567
RAMNT_5,99.990567
RDATE_3,99.746363
RAMNT_3,99.746363
RDATE_4,99.705488
...,...
ADATE_16,21.343227
ADATE_14,19.774242
ADATE_9,11.785729
ADATE_11,10.923154


Data Cleaning

In [16]:
#After checking the internet
#I have decided to check the internet, what to do with dataframes with a lot of columns. 
#I saw that I could drop columns where the data in the columns are more than 10%

In [17]:
less_ten_10_percent = list(nulls[nulls['percentage'] < 10].index)

In [18]:
#This are the columns I want to keep
less_ten_10_percent

['ODATEDW',
 'OSOURCE',
 'TCODE',
 'STATE',
 'ZIP',
 'MAILCODE',
 'PVASTATE',
 'DOB',
 'NOEXCH',
 'RECINHSE',
 'RECP3',
 'RECPGVG',
 'RECSWEEP',
 'MDMAUD',
 'DOMAIN',
 'CLUSTER',
 'AGEFLAG',
 'HOMEOWNR',
 'CHILD03',
 'CHILD07',
 'CHILD12',
 'CHILD18',
 'GENDER',
 'HIT',
 'DATASRCE',
 'MALEMILI',
 'MALEVET',
 'VIETVETS',
 'WWIIVETS',
 'LOCALGOV',
 'STATEGOV',
 'FEDGOV',
 'SOLP3',
 'SOLIH',
 'MAJOR',
 'GEOCODE',
 'COLLECT1',
 'VETERANS',
 'BIBLE',
 'CATLG',
 'HOMEE',
 'PETS',
 'CDPLAY',
 'STEREO',
 'PCOWNERS',
 'PHOTO',
 'CRAFTS',
 'FISHER',
 'GARDENIN',
 'BOATS',
 'WALKER',
 'KIDSTUFF',
 'CARDS',
 'PLATES',
 'LIFESRC',
 'PEPSTRFL',
 'POP901',
 'POP902',
 'POP903',
 'POP90C1',
 'POP90C2',
 'POP90C3',
 'POP90C4',
 'POP90C5',
 'ETH1',
 'ETH2',
 'ETH3',
 'ETH4',
 'ETH5',
 'ETH6',
 'ETH7',
 'ETH8',
 'ETH9',
 'ETH10',
 'ETH11',
 'ETH12',
 'ETH13',
 'ETH14',
 'ETH15',
 'ETH16',
 'AGE901',
 'AGE902',
 'AGE903',
 'AGE904',
 'AGE905',
 'AGE906',
 'AGE907',
 'CHIL1',
 'CHIL2',
 'CHIL3',
 'AGEC1',


In [19]:
#Checking how many columns there are left
len(less_ten_10_percent)

400

In [20]:
#Putting the data back in a dataframe
data = file[less_ten_10_percent]

In [21]:
data

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,,,3712,0,,...,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,BOA,1,CA,91326,,,5202,0,,...,0.0,0,L,2,G,X,X,X,1.0,A
2,9001,AMH,1,NC,27017,,,0,0,,...,0.0,1,L,4,E,X,X,X,60.0,C
3,8701,BRY,0,CA,95953,,,2801,0,,...,0.0,1,L,4,E,X,X,X,41.0,C
4,8601,,0,FL,33176,,,2001,0,X,...,0.0,1,L,2,F,X,X,X,26.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,9601,ASE,1,AK,99504,,,0,0,,...,0.0,0,L,1,G,X,X,X,12.0,C
95408,9601,DCD,1,TX,77379,,,5001,0,,...,0.0,1,L,1,F,X,X,X,2.0,A
95409,9501,MBC,1,MI,48910,,,3801,0,,...,0.0,1,L,3,E,X,X,X,34.0,B
95410,8601,PRV,0,CA,91320,,,4005,0,X,...,18.0,1,L,4,F,X,X,X,11.0,A


In [22]:
#I am already defening the X variable
X = data.drop(columns = ['TARGET_B','TARGET_D','CONTROLN', 'OSOURCE'])

In [23]:
X

,ODATEDW,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,RECP3,...,AVGGIFT,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,0,IL,61081,,,3712,0,,,...,7.741935,0,L,4,E,X,X,X,39.0,C
1,9401,1,CA,91326,,,5202,0,,,...,15.666667,0,L,2,G,X,X,X,1.0,A
2,9001,1,NC,27017,,,0,0,,,...,7.481481,1,L,4,E,X,X,X,60.0,C
3,8701,0,CA,95953,,,2801,0,,,...,6.812500,1,L,4,E,X,X,X,41.0,C
4,8601,0,FL,33176,,,2001,0,X,X,...,6.864865,1,L,2,F,X,X,X,26.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,9601,1,AK,99504,,,0,0,,,...,25.000000,0,L,1,G,X,X,X,12.0,C
95408,9601,1,TX,77379,,,5001,0,,,...,20.000000,1,L,1,F,X,X,X,2.0,A
95409,9501,1,MI,48910,,,3801,0,,X,...,8.285714,1,L,3,E,X,X,X,34.0,B
95410,8601,0,CA,91320,,,4005,0,X,,...,12.146341,1,L,4,F,X,X,X,11.0,A


In [24]:
#I am splitting the numerical data from the categorical data 
xnum = X.select_dtypes(include= np.number)
xcat = X.select_dtypes(include='object')

In [25]:
#I am filling the nulls 
xnum = xnum.fillna(0)
xcat = xcat.fillna('')

In [26]:
xcat.shape

(95412, 73)

In [27]:
#Here I am checking the correlation with the xcat, to drop the columns with low correlation 
cols_to_remove = []

i = 0
for col1 in xcat.columns:
    i += 1
    print(i, col1)
    for col2 in xcat.columns:
        if col1 != col2  and col1 not in cols_to_remove and col2 not in cols_to_remove:
            data_crosstab = pd.crosstab(data[col1], data[col2], margins=True)
            _, p_value, _, _ = chi2_contingency(data_crosstab)

            if p_value <= .01:
                data_crosstab = pd.crosstab(data[col1], data['TARGET_B'])
                _, p_value1, _, _ = chi2_contingency(data_crosstab)
                data_crosstab = pd.crosstab(data[col2], data['TARGET_B'])
                _, p_value2, _, _ = chi2_contingency(data_crosstab)

                if p_value1 > p_value2 and p_value1 > .1:
                    cols_to_remove.append(col1) #name
                    print(col2, p_value1)

1 STATE
2 ZIP
STATE 0.9958545269431777
3 MAILCODE
4 PVASTATE
STATE 0.8712387319713483
5 NOEXCH
MAILCODE 0.9610197835149104
6 RECINHSE
7 RECP3
8 RECPGVG
RECINHSE 0.2466642159207489
9 RECSWEEP
STATE 0.6009808546810607
10 MDMAUD
RECINHSE 0.21901127668839243
11 DOMAIN
12 CLUSTER
13 AGEFLAG
14 HOMEOWNR
15 CHILD03
STATE 0.20247480497655432
16 CHILD07
RECINHSE 0.5022187873299642
17 CHILD12
RECINHSE 0.2638372430893398
18 CHILD18
STATE 0.5893286221985807
19 GENDER
20 DATASRCE
STATE 0.251030736726543
21 SOLP3
22 SOLIH
23 MAJOR
RECINHSE 0.910358311652111
24 GEOCODE
25 COLLECT1
26 VETERANS
27 BIBLE
28 CATLG
29 HOMEE
DOMAIN 0.49143846696979454
30 PETS
31 CDPLAY
STATE 0.2801031645855597
32 STEREO
MAILCODE 0.8594281715830145
33 PCOWNERS
STATE 0.9434806697379569
34 PHOTO
35 CRAFTS
36 FISHER
STATE 0.7968480566406873
37 GARDENIN
38 BOATS
STATE 0.3282331689225323
39 WALKER
40 KIDSTUFF
RECINHSE 0.6787292545054087
41 CARDS
AGEFLAG 0.27679759764810186
42 PLATES
AGEFLAG 0.24080332333106783
43 LIFESRC
44 PEPS

In [28]:
#Checking how many columns there are to remove
len(cols_to_remove)


25

In [29]:
#chechking the columns, in xcat, they are 74
xcat.shape

(95412, 73)

In [30]:
#Removing the columns from the columns from the x_cat
x_cat = xcat.drop(columns = ['ZIP','PVASTATE','NOEXCH','RECPGVG','RECSWEEP','MDMAUD','CHILD03','CHILD07','CHILD12','CHILD18','DATASRCE','MAJOR','HOMEE','CDPLAY','STEREO','PCOWNERS','FISHER','BOATS','KIDSTUFF','CARDS','PLATES','MDMAUD_R','MDMAUD_F','MDMAUD_A','GEOCODE2'], axis=1)

In [31]:
x_cat

,STATE,MAILCODE,RECINHSE,RECP3,DOMAIN,CLUSTER,AGEFLAG,HOMEOWNR,GENDER,SOLP3,...,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24,RFA_2R,RFA_2A
0,IL,,,,T2,36,,,F,,...,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,L,E
1,CA,,,,S1,14,E,H,M,,...,,,N1E,N1E,N1E,N1E,,F1E,L,G
2,NC,,,,R2,43,,U,M,,...,,S4D,S4D,,,S4D,S4D,S3D,L,E
3,CA,,,,R2,44,E,U,F,,...,S2D,S2D,A1D,A1D,A1D,A1D,,,L,E
4,FL,,X,X,S2,16,E,H,F,,...,A2D,A2D,A3D,A3D,A3D,I4E,A3D,A3D,L,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,AK,,,,C2,27,,,M,,...,,,,,,,,,L,G
95408,TX,,,,C1,24,E,H,M,,...,,,,,,,,,L,F
95409,MI,,,X,C3,30,,,M,,...,,F1D,,,P1D,P1D,,,L,E
95410,CA,,X,,C1,24,E,H,F,,...,S2F,S2F,A1F,A1F,A1F,A1F,S2F,S3F,L,F


In [32]:
#checking the columns, in x_cat, the are 49 (74-25 = 49)
x_cat.shape

for c in x_cat:
    if len(x_cat[c].unique()) > 100:
        print(c)
len(x_cat['OSOURCE'].unique())

RFA_6
RFA_7
RFA_8
RFA_9
RFA_11
RFA_12
RFA_16
RFA_17
RFA_18
RFA_19
RFA_21
RFA_22


KeyError: 'OSOURCE'

In [33]:
#The numerical data:
#scaling the data
#Using PCA to put the columns in a smaller set of columns 

scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(xnum)

pca = PCA(n_components=.95)
pca.fit(x_scaled, data['TARGET_B'])
xnum = pca.transform(xnum)

xnum.shape

(95412, 67)

In [34]:
# I want to concate x_cat and xnum, I want to see how they look like separate
x_cat

,STATE,MAILCODE,RECINHSE,RECP3,DOMAIN,CLUSTER,AGEFLAG,HOMEOWNR,GENDER,SOLP3,...,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24,RFA_2R,RFA_2A
0,IL,,,,T2,36,,,F,,...,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,L,E
1,CA,,,,S1,14,E,H,M,,...,,,N1E,N1E,N1E,N1E,,F1E,L,G
2,NC,,,,R2,43,,U,M,,...,,S4D,S4D,,,S4D,S4D,S3D,L,E
3,CA,,,,R2,44,E,U,F,,...,S2D,S2D,A1D,A1D,A1D,A1D,,,L,E
4,FL,,X,X,S2,16,E,H,F,,...,A2D,A2D,A3D,A3D,A3D,I4E,A3D,A3D,L,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,AK,,,,C2,27,,,M,,...,,,,,,,,,L,G
95408,TX,,,,C1,24,E,H,M,,...,,,,,,,,,L,F
95409,MI,,,X,C3,30,,,M,,...,,F1D,,,P1D,P1D,,,L,E
95410,CA,,X,,C1,24,E,H,F,,...,S2F,S2F,A1F,A1F,A1F,A1F,S2F,S3F,L,F


In [35]:
dummy_cat = pd.get_dummies(x_cat)
dummy_cat.shape

(95412, 2290)

In [36]:
scaler = MinMaxScaler()
x_scaled_cat = scaler.fit_transform(dummy_cat)

pca = PCA(n_components=.95)
pca.fit(x_scaled_cat, data['TARGET_B'])
dummy_cat = pca.transform(x_scaled_cat)
dummy_cat.shape

(95412, 540)

In [37]:
xnum

array([[  831.88322727,  -646.8618953 ,  -363.95531641, ...,
          676.66063552,  2064.07174067,  -632.2305592 ],
       [ 3574.42415852, -1134.82991987,  -210.07112523, ...,
         1459.62266889,  2928.57942188,   137.53503803],
       [  681.97323327,  -547.51104097,   -62.10406478, ...,
         1677.90377228,  2317.46486733,   202.28531008],
       ...,
       [ 1368.55096204,  -367.42454848,  -195.87731252, ...,
          596.46384559,  2328.66613834,  -871.68894553],
       [ 2770.20504128,  -537.11849453,   -92.37382704, ...,
          792.33146357,  1922.31697309,  -339.82300727],
       [ 2407.24142578,  -449.2485689 ,   114.89424986, ...,
         1206.67970445,  2230.77455508,  -104.75087664]])

In [38]:
# Here I am making a dataframe of xnum, so I can concate it with with x_cat
df = pd.DataFrame(xnum)

In [39]:
df.dtypes

0     float64
1     float64
2     float64
3     float64
4     float64
       ...   
62    float64
63    float64
64    float64
65    float64
66    float64
Length: 67, dtype: object

In [40]:
dummy_cat = pd.DataFrame(dummy_cat)

In [41]:
dummy_cat

,0,1,2,3,4,5,6,7,8,9,...,530,531,532,533,534,535,536,537,538,539
0,-1.543898,-1.414390,-1.182505,-0.913462,-1.019044,-0.186667,0.458338,0.442526,0.042676,-0.057756,...,0.055177,0.058121,0.016822,-0.056233,-0.055087,0.007557,-0.008825,-0.036627,0.016017,0.082159
1,0.118673,-0.286203,0.037046,0.394775,1.994977,0.372205,-0.605139,-0.683514,0.107954,0.061136,...,0.020850,0.008793,0.002238,-0.010681,0.049743,0.065684,-0.025363,-0.030930,-0.004446,-0.015777
2,-0.582553,-1.445814,-0.959750,-0.740996,-1.104002,-0.566962,0.356428,-0.978498,-0.136501,0.092563,...,-0.135746,0.069506,-0.141231,-0.040676,0.057974,0.053992,0.123118,0.051522,-0.030138,0.116640
3,-0.908226,-1.223641,-0.441972,-0.822089,-0.632321,0.125704,-0.683564,0.346862,0.168112,-0.333127,...,-0.018941,-0.025626,-0.028587,-0.018650,0.068444,-0.033556,0.028733,0.031233,-0.019987,-0.081835
4,-0.893818,0.359498,1.994094,-0.306228,0.522387,1.086882,0.608042,0.736720,-0.539454,1.112968,...,-0.047736,-0.013447,0.021439,0.078648,0.003598,0.050231,-0.010456,0.005339,0.085180,0.138701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,2.812180,-0.992399,-1.024800,0.648204,0.406791,-0.721104,0.749952,-0.554647,0.570987,0.301602,...,-0.016459,-0.006181,0.025597,0.002865,-0.031965,0.010783,-0.007026,-0.029539,0.007391,-0.014890
95408,2.970057,0.118872,-0.022766,-0.071505,0.297184,-0.843509,-0.807151,-0.756627,-0.336047,0.358324,...,0.001280,0.047298,-0.063582,-0.011937,0.017651,0.010413,-0.018632,0.048859,0.009844,0.033587
95409,-0.048154,-1.252510,-0.998494,-0.501840,-0.747076,0.579754,0.594215,-1.011395,0.679233,-0.419613,...,0.095174,0.197797,0.122209,-0.000735,0.120857,-0.158416,-0.051092,0.001329,-0.231483,0.176777
95410,-1.582883,-0.147436,0.139371,-0.026801,-0.757344,-0.551932,-0.883893,0.369235,-0.907509,0.809890,...,-0.063948,0.076816,0.026445,-0.034682,-0.050798,0.052023,-0.034453,0.098954,-0.009225,-0.044114


In [42]:
dummy_cat.dtypes

0      float64
1      float64
2      float64
3      float64
4      float64
        ...   
535    float64
536    float64
537    float64
538    float64
539    float64
Length: 540, dtype: object

In [43]:
f = dummy_cat.astype(float)
f.dtypes

0      float64
1      float64
2      float64
3      float64
4      float64
        ...   
535    float64
536    float64
537    float64
538    float64
539    float64
Length: 540, dtype: object

In [44]:
#Here I concat the dataframes
x = pd.concat([df, f], axis=1)
x

,0,1,2,3,4,5,6,7,8,9,...,530,531,532,533,534,535,536,537,538,539
0,831.883227,-646.861895,-363.955316,-182.229876,-400.367679,-381.638509,265.626351,277.234640,21.686036,3375.632218,...,0.055177,0.058121,0.016822,-0.056233,-0.055087,0.007557,-0.008825,-0.036627,0.016017,0.082159
1,3574.424159,-1134.829920,-210.071125,1506.846801,-710.712659,-1000.928500,697.962871,-156.237400,-541.669607,2921.697086,...,0.020850,0.008793,0.002238,-0.010681,0.049743,0.065684,-0.025363,-0.030930,-0.004446,-0.015777
2,681.973233,-547.511041,-62.104065,-152.397865,-532.179443,-663.551389,-36.567874,297.875608,-238.958811,3497.992065,...,-0.135746,0.069506,-0.141231,-0.040676,0.057974,0.053992,0.123118,0.051522,-0.030138,0.116640
3,2248.176026,-371.333556,17.341024,153.031358,-559.494031,202.556490,61.571859,-1623.052607,-1222.283285,3686.672549,...,-0.018941,-0.025626,-0.028587,-0.018650,0.068444,-0.033556,0.028733,0.031233,-0.019987,-0.081835
4,1424.241454,-326.772925,-135.899676,-236.083293,-504.559563,9.072990,-37.746297,-853.493295,-824.684647,3658.558861,...,-0.047736,-0.013447,0.021439,0.078648,0.003598,0.050231,-0.010456,0.005339,0.085180,0.138701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,1466.972344,-537.481623,417.465311,-241.452684,-1178.093742,-1058.511164,-443.431369,560.561072,-127.990495,-1053.981987,...,-0.016459,-0.006181,0.025597,0.002865,-0.031965,0.010783,-0.007026,-0.029539,0.007391,-0.014890
95408,2345.345493,-803.998002,73.513108,406.864479,-728.035887,-55.330810,184.177274,-172.615589,535.689080,-2955.636037,...,0.001280,0.047298,-0.063582,-0.011937,0.017651,0.010413,-0.018632,0.048859,0.009844,0.033587
95409,1368.550962,-367.424548,-195.877313,-325.880815,-388.317946,194.174788,-14.036008,-774.801858,-290.839044,2278.121890,...,0.095174,0.197797,0.122209,-0.000735,0.120857,-0.158416,-0.051092,0.001329,-0.231483,0.176777
95410,2770.205041,-537.118495,-92.373827,352.557972,-640.184514,-15.855259,191.368300,-1379.473664,-1192.886720,3617.889383,...,-0.063948,0.076816,0.026445,-0.034682,-0.050798,0.052023,-0.034453,0.098954,-0.009225,-0.044114


In [45]:
x.dtypes

0      float64
1      float64
2      float64
3      float64
4      float64
        ...   
535    float64
536    float64
537    float64
538    float64
539    float64
Length: 607, dtype: object

In [46]:
x.astype(float)

,0,1,2,3,4,5,6,7,8,9,...,530,531,532,533,534,535,536,537,538,539
0,831.883227,-646.861895,-363.955316,-182.229876,-400.367679,-381.638509,265.626351,277.234640,21.686036,3375.632218,...,0.055177,0.058121,0.016822,-0.056233,-0.055087,0.007557,-0.008825,-0.036627,0.016017,0.082159
1,3574.424159,-1134.829920,-210.071125,1506.846801,-710.712659,-1000.928500,697.962871,-156.237400,-541.669607,2921.697086,...,0.020850,0.008793,0.002238,-0.010681,0.049743,0.065684,-0.025363,-0.030930,-0.004446,-0.015777
2,681.973233,-547.511041,-62.104065,-152.397865,-532.179443,-663.551389,-36.567874,297.875608,-238.958811,3497.992065,...,-0.135746,0.069506,-0.141231,-0.040676,0.057974,0.053992,0.123118,0.051522,-0.030138,0.116640
3,2248.176026,-371.333556,17.341024,153.031358,-559.494031,202.556490,61.571859,-1623.052607,-1222.283285,3686.672549,...,-0.018941,-0.025626,-0.028587,-0.018650,0.068444,-0.033556,0.028733,0.031233,-0.019987,-0.081835
4,1424.241454,-326.772925,-135.899676,-236.083293,-504.559563,9.072990,-37.746297,-853.493295,-824.684647,3658.558861,...,-0.047736,-0.013447,0.021439,0.078648,0.003598,0.050231,-0.010456,0.005339,0.085180,0.138701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,1466.972344,-537.481623,417.465311,-241.452684,-1178.093742,-1058.511164,-443.431369,560.561072,-127.990495,-1053.981987,...,-0.016459,-0.006181,0.025597,0.002865,-0.031965,0.010783,-0.007026,-0.029539,0.007391,-0.014890
95408,2345.345493,-803.998002,73.513108,406.864479,-728.035887,-55.330810,184.177274,-172.615589,535.689080,-2955.636037,...,0.001280,0.047298,-0.063582,-0.011937,0.017651,0.010413,-0.018632,0.048859,0.009844,0.033587
95409,1368.550962,-367.424548,-195.877313,-325.880815,-388.317946,194.174788,-14.036008,-774.801858,-290.839044,2278.121890,...,0.095174,0.197797,0.122209,-0.000735,0.120857,-0.158416,-0.051092,0.001329,-0.231483,0.176777
95410,2770.205041,-537.118495,-92.373827,352.557972,-640.184514,-15.855259,191.368300,-1379.473664,-1192.886720,3617.889383,...,-0.063948,0.076816,0.026445,-0.034682,-0.050798,0.052023,-0.034453,0.098954,-0.009225,-0.044114


In [47]:
#Because I data['TARGET_D'] is not integreted in the dataframe 
#I concate it with the rest of the data

x = x.astype(float)
x.dtypes
set(x.dtypes)
x.shape

(95412, 607)

In [48]:
y = data['TARGET_B']
y = data['TARGET_B'].astype(float)

In [49]:
y.dtypes

dtype('float64')

Making the models

In [50]:
#Starting with the benchmark

In [51]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

classification = LogisticRegression(random_state=42, max_iter=10000)
classification.fit(X_train, y_train)

LogisticRegression(max_iter=10000, random_state=42)

In [52]:
classification.score(X_test, y_test)

0.9487501965099827

In [53]:
from sklearn.metrics import classification_report

In [54]:
predictions = classification.predict(X_test)

In [55]:
data.loc[y_test.index]['TARGET_D'].unique()

array([  0. ,  10. ,  15. ,   9. ,   7. ,  20. ,  25. , 100. ,  21. ,
         8. ,   5. ,  30. ,  17. ,  14. ,  13. ,   6. ,  46. ,   2. ,
        50. ,  16. ,  11. ,   4. ,  23. ,   3. ,  18. ,  26. ,  27. ,
         1. ,  12. ,  41. ,  35. ,  32. ,  22. ,  19. ,  40. ,  12.5,
         7.5, 200. ,  36. ,  28. ,  24. ,  38. ,  45. , 102. ])

In [56]:
df = pd.DataFrame()
df['send_mail'] = predictions
df['TARGET_D'] = file.loc[y_test.index]['TARGET_D'].to_list()
df['TARGET_B'] = file.loc[y_test.index]['TARGET_B'].to_list()

total_amount_spent_on_mail = len(df[df['send_mail'] > 0]) * 0.68
total_amount_spent_on_mail_everyone = len(df) * 0.68
total_missed_donations = sum(df[(df['send_mail'] == 0) & (df['TARGET_D'] > 0)]['TARGET_D'])
total_extra_spent_on_mail = round(len(df[(df['send_mail'] == 1) & (df['TARGET_B'] == 0)]) * 0.68, 2)

savings = round(total_amount_spent_on_mail_everyone - total_missed_donations - total_amount_spent_on_mail, 2)

if savings <= 0:
    print(f'This model is not worth it at the moment. Just mail everyone! Savings: {savings}')
else:
    print(f'''This model saved ${savings} in mails.
        Missed ${total_missed_donations} in donations and wasted ${total_extra_spent_on_mail} in mails not responded.
        You made: ${sum(df['TARGET_D']) - total_amount_spent_on_mail}
           ''')

This model is not worth it at the moment. Just mail everyone! Savings: -2466.56


In [57]:
#Before I will make model 2 I will scale the data first
#Upsampling

In [58]:
# Using the scaled data and do something with the class_weight={1:3}
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

from imblearn.over_sampling import SMOTE

smote = SMOTE()

X_sm, y_sm = smote.fit_resample (np.array(X_train), np.array(y_train))

classification = LogisticRegression(random_state=42, class_weight={1:3}, max_iter=10000)
classification.fit(X_sm, y_sm)

LogisticRegression(class_weight={1: 3}, max_iter=10000, random_state=42)

In [67]:
predictions = classification.predict(X_test)

In [68]:
# This is a better model
df = pd.DataFrame()
df['send_mail'] = predictions
df['TARGET_D'] = file.loc[y_test.index]['TARGET_D'].to_list()
df['TARGET_B'] = file.loc[y_test.index]['TARGET_B'].to_list()

total_amount_spent_on_mail = len(df[df['send_mail'] > 0]) * 0.68
total_amount_spent_on_mail_everyone = len(df) * 0.68
total_missed_donations = sum(df[(df['send_mail'] == 0) & (df['TARGET_D'] > 0)]['TARGET_D'])
total_extra_spent_on_mail = round(len(df[(df['send_mail'] == 1) & (df['TARGET_B'] == 0)]) * 0.68, 2)

savings = round(total_amount_spent_on_mail_everyone - total_missed_donations - total_amount_spent_on_mail, 2)

if savings <= 0:
    print(f'This model is not worth it at the moment. Just mail everyone! Savings: {savings}')
else:
    print(f'''This model saved ${savings} in mails.
        Missed ${total_missed_donations} in donations and wasted ${total_extra_spent_on_mail} in mails not responded.
        You made: ${sum(df['TARGET_D']) - total_amount_spent_on_mail}
           ''')

This model is not worth it at the moment. Just mail everyone! Savings: -175.68


In [69]:
#Trying the RandomForestClassifier and use the data that is sampled
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report

model = RandomForestClassifier()

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

smote = SMOTE()

X_sm, y_sm = smote.fit_resample (np.array(X_train), np.array(y_train))

model.fit(X_sm,y_sm)

predictions = model.predict(X_test)

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97     27200
         1.0       0.10      0.00      0.00      1424

    accuracy                           0.95     28624
   macro avg       0.53      0.50      0.49     28624
weighted avg       0.91      0.95      0.93     28624



In [70]:
# This model is not worth using 
df = pd.DataFrame()
df['send_mail'] = predictions
df['TARGET_D'] = file.loc[y_test.index]['TARGET_D'].to_list()
df['TARGET_B'] = file.loc[y_test.index]['TARGET_B'].to_list()

total_amount_spent_on_mail = len(df[df['send_mail'] > 0]) * 0.68
total_amount_spent_on_mail_everyone = len(df) * 0.68
total_missed_donations = sum(df[(df['send_mail'] == 0) & (df['TARGET_D'] > 0)]['TARGET_D'])
total_extra_spent_on_mail = round(len(df[(df['send_mail'] == 1) & (df['TARGET_B'] == 0)]) * 0.68, 2)

savings = round(total_amount_spent_on_mail_everyone - total_missed_donations - total_amount_spent_on_mail, 2)

if savings <= 0:
    print(f'This model is not worth it at the moment. Just mail everyone! Savings: {savings}')
else:
    print(f'''This model saved ${savings} in mails.
        Missed ${total_missed_donations} in donations and wasted ${total_extra_spent_on_mail} in mails not responded.
        You made: ${sum(df['TARGET_D']) - total_amount_spent_on_mail}
           ''')

This model is not worth it at the moment. Just mail everyone! Savings: -2966.33


In [72]:
from sklearn.neighbors import KNeighborsClassifier
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25,
                                                    stratify=y, random_state=0)

smote = SMOTE()

X_sm, y_sm = smote.fit_resample (np.array(X_train), np.array(y_train))

knn = KNeighborsClassifier()
knn.fit(X_sm,y_sm)
predictions = knn.predict(X_test)

In [73]:
df = pd.DataFrame()
df['send_mail'] = predictions
df['TARGET_D'] = file.loc[y_test.index]['TARGET_D'].to_list()
df['TARGET_B'] = file.loc[y_test.index]['TARGET_B'].to_list()

total_amount_spent_on_mail = len(df[df['send_mail'] > 0]) * 0.68
total_amount_spent_on_mail_everyone = len(df) * 0.68
total_missed_donations = sum(df[(df['send_mail'] == 0) & (df['TARGET_D'] > 0)]['TARGET_D'])
total_extra_spent_on_mail = round(len(df[(df['send_mail'] == 1) & (df['TARGET_B'] == 0)]) * 0.68, 2)

savings = round(total_amount_spent_on_mail_everyone - total_missed_donations - total_amount_spent_on_mail, 2)

if savings <= 0:
    print(f'This model is not worth it at the moment. Just mail everyone! Savings: {savings}')
else:
    print(f'''This model saved ${savings} in mails.
        Missed ${total_missed_donations} in donations and wasted ${total_extra_spent_on_mail} in mails not responded.
        You made: ${sum(df['TARGET_D']) - total_amount_spent_on_mail}
           ''')

This model is not worth it at the moment. Just mail everyone! Savings: -1684.06


In [ ]:
#Conclussion: No one of my models I use. I could best mail everyone 